# Motivation
Purpose was to create an interactive map, so we can easily identify customers based on the location and National Taxonomy of Exempt Entities (NTEE) Codes. The NtEE code is used by IRS to classify each tax exempt organization based on the main activity. You can find more information about this here: https://nccs.urban.org/publication/irs-activity-codes. I have also included address and phone numbers to help with navigation in the future. The scope of the project is to only focus on charities in Hoboken.

In [1]:
# Loading packages
from geopy.geocoders import Nominatim
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
import numpy as np
from geopy.extra.rate_limiter import RateLimiter

# Load Data

In [2]:
# Load the data that was cleaned based on the original open990 database.
query=''' SELECT *
FROM hoboken_phone_address'''
e = sa.create_engine(r'sqlite:///D:\Program_practices\Charity\open990database.db')
df= pd.read_sql_query(query, e)
df.head(5)

,ein,name_org,tax_yr,phone,website,address,formation_yr,gross_receipts,ntee_code_nccs,ntee_description_nccs,voting_memb_ct,employee_tot_ct,volunteer_tot_ct,gross_receipts:1,rev_tot_curyr,rev_less_expense_curyr,local_chapt
0,203671056,Symposia Community Bookstore,2013,+1 (201) 963-0909,None,510 Washington Street,2007,192350,P29,Thrift Shops,6,2,None,192350,192350,-1498,no
1,203671056,Symposia Community Bookstore,2015,+1 (201) 963-0909,None,510 Washington Street,2007,180066,P29,Thrift Shops,6,0,None,180066,180066,-1937,no
2,222209099,STEVENS COOPERATIVE SCHOOL,2010,+1 (201) 792-3688,WWW.STEVENSCOOP.ORG,301 GARDEN STREET,1949,5935065,B24,"Primary, Elementary Schools",15,118,0,5935065,5858009,276650,no
3,221763570,CHURCH TOWERS URBAN RENEWAL CORP C/O TOTAL PRO...,2013,+1 (201) 659-0604,None,10 CHURCH TOWERS,1965,4347916,S20,"Community, Neighborhood Development, Improveme...",2,33,None,4347916,4347916,-121051,no
4,136271999,ASSOCIATION OF THE GRADUATES OF THE COLUMBIA C...,2014,+1 (201) 993-9480,None,1015 WASHINGTON STREET UNIT 50,None,248122,B83,"Student Sororities, Fraternities",14,0,None,248122,248122,33713,no


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 17 columns):
ein                       175 non-null int64
name_org                  175 non-null object
tax_yr                    175 non-null int64
phone                     166 non-null object
website                   93 non-null object
address                   175 non-null object
formation_yr              163 non-null object
gross_receipts            175 non-null int64
ntee_code_nccs            128 non-null object
ntee_description_nccs     128 non-null object
voting_memb_ct            175 non-null int64
employee_tot_ct           175 non-null int64
volunteer_tot_ct          108 non-null object
gross_receipts:1          175 non-null int64
rev_tot_curyr             175 non-null int64
rev_less_expense_curyr    175 non-null int64
local_chapt               175 non-null object
dtypes: int64(8), object(9)
memory usage: 23.4+ KB


# Data Cleaning
The original database has multiple tax years. I will only be taking the most recent one, assuming that this is the address and phone number are up-to-date. What I have noticed based on previous work, however, is that NTEE code is sometimes not consistent. When it is not consistent, I took the most recent code. Most of the time, it seems like NTEE code was left out by human mistake and was left blank. In this case, I created a key for the NTEE code to be added based on previous tax return records using .fillna method. 

In [4]:
query=''' SELECT *
FROM hoboken_merger'''

e = sa.create_engine(r'sqlite:///D:\Program_practices\Charity\open990database.db')
df_merg= pd.read_sql_query(query, e)
key=df_merg[['ein', 'ntee_code_nccs']]
#%%
df=pd.merge(df,key, on='ein', how='left')
df = df.drop_duplicates(subset='ein', keep="last")
df['ntee_code_nccs_x']=df['ntee_code_nccs_x'].fillna(df['ntee_code_nccs_y'])
df=df.drop(columns='ntee_code_nccs_y')
df.head(5)

,ein,name_org,tax_yr,phone,website,address,formation_yr,gross_receipts,ntee_code_nccs_x,ntee_description_nccs,voting_memb_ct,employee_tot_ct,volunteer_tot_ct,gross_receipts:1,rev_tot_curyr,rev_less_expense_curyr,local_chapt
11,226052626,STEVENS ALUMNI ASSOCIATION,2011,+1 (201) 216-5163,WWW.ALUMNI.STEVENS-TECH.EDU,CASTLE POINT ON THE HUDSON,1958,956073,B84,Alumni Associations,25,0,300,956073,705616,-22150,no
14,222891851,Columbian Arms Inc,2015,None,None,514 MADISON STREET,1987,896454,L21,Public Housing Facilities,9,0,None,896454,-40166,-40166,no
64,223517040,HOBOKEN CHARTER SCHOOL INC,2016,+1 (201) 963-0222,WWW.HOBOKENCS.ORG,713 WASHINGTON STREET,1997,5346175,None,None,9,76,None,5346175,5346175,276176,no
96,221801849,HOPES COMMUNITY ACTION PARTNERSHIP,2016,+1 (855) 654-6737,WWW.HOPES.ORG,301 GARDEN STREET,1964,16748872,S20,None,17,307,788,16748872,15513189,1916145,no
107,200596839,Party With Purpose Inc,2014,+1 (201) 988-7175,None,PO Box 1045,2002,183780,T31,Community Foundations,22,0,100,183780,183780,55830,no


To plot this on a map, the address has to be converted into latitude and longitude. You can use Google API, but I used geopy.geocoders

In [5]:
locator = Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [6]:
df['address']=df['address']
df['address']=df['address'].str.title()
df['address']

11                            Castle Point On The Hudson
14                                    514 Madison Street
64                                 713 Washington Street
96                                     301 Garden Street
107                                          Po Box 1045
110                                       308 Willow Ave
111                       25 Fifth Street The Fieldhouse
155                                    404 Hudson Street
177                               601-603 Jackson Street
213                                   1460 Garden Street
224                                      601 Park Avenue
232                              106 Hudson St 2Nd Floor
249                                    308 Willow Avenue
255                     C/O P Capra 51 Harrison St No 5D
259                                          Po Box 3296
265                                    301 Garden Street
278                                    301 Garden Street
293                            

As you can see, the detailed information about the building (eg, unit, room, and floor numbers) is included. Since we don't need these numbers to locate each organization and these numbers will likely interfere with geocoding, these will be removed below. In addition, some of the addresses are listed as PO box, and they will not be coded properly. Thus, adopting the name of the company will be better. 

In [7]:
import re
df['address_code']=df.apply(lambda df: df['name_org'] if 'Po Box' in df['address'] else df['address'], axis=1)
df['address_code']=df['address_code'].str.title()
df['address_code']

11                            Castle Point On The Hudson
14                                    514 Madison Street
64                                 713 Washington Street
96                                     301 Garden Street
107                               Party With Purpose Inc
110                                       308 Willow Ave
111                       25 Fifth Street The Fieldhouse
155                                    404 Hudson Street
177                               601-603 Jackson Street
213                                   1460 Garden Street
224                                      601 Park Avenue
232                              106 Hudson St 2Nd Floor
249                                    308 Willow Avenue
255                     C/O P Capra 51 Harrison St No 5D
259                        Hoboken Historical Museum Inc
265                                    301 Garden Street
278                                    301 Garden Street
293                            

Let's delete all the irrelevant numbers (eg, unit number) from the end. I hard coded since we do not have that many endings, but future code with a larger data set can be improved. I used https://regex101.com/ to optimize the regular expressions.

In [8]:
df['address_code']=df['address_code'].apply(lambda address: 
                                       re.findall(r'(.*(?:Street|St|Ave|Avenue|Hudson|Highway|Terrace|Rd|Towers|Rm/St|Chapter|Inc))',address)[0])

df['address_code']

11            Castle Point On The Hudson
14                    514 Madison Street
64                 713 Washington Street
96                     301 Garden Street
107               Party With Purpose Inc
110                       308 Willow Ave
111                      25 Fifth Street
155                    404 Hudson Street
177               601-603 Jackson Street
213                   1460 Garden Street
224                         601 Park Ave
232                        106 Hudson St
249                       308 Willow Ave
255           C/O P Capra 51 Harrison St
259        Hoboken Historical Museum Inc
265                    301 Garden Street
278                    301 Garden Street
293                          251 10Th St
294             926 Castle Point Terrace
297                     700 First Street
304               Castle Point On Hudson
306                       422 Willow Ave
312                     124 Grand Street
314                713 Washington Street
316             

In [9]:
df['address_code']=df['address_code']+ ', Hoboken, Nj'
df['address_code']

11            Castle Point On The Hudson, Hoboken, Nj
14                    514 Madison Street, Hoboken, Nj
64                 713 Washington Street, Hoboken, Nj
96                     301 Garden Street, Hoboken, Nj
107               Party With Purpose Inc, Hoboken, Nj
110                       308 Willow Ave, Hoboken, Nj
111                      25 Fifth Street, Hoboken, Nj
155                    404 Hudson Street, Hoboken, Nj
177               601-603 Jackson Street, Hoboken, Nj
213                   1460 Garden Street, Hoboken, Nj
224                         601 Park Ave, Hoboken, Nj
232                        106 Hudson St, Hoboken, Nj
249                       308 Willow Ave, Hoboken, Nj
255           C/O P Capra 51 Harrison St, Hoboken, Nj
259        Hoboken Historical Museum Inc, Hoboken, Nj
265                    301 Garden Street, Hoboken, Nj
278                    301 Garden Street, Hoboken, Nj
293                          251 10Th St, Hoboken, Nj
294             926 Castle P

Finally, geocoding the address....

In [10]:
df['location'] = df['address_code'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df['latitude']=df['point'].str[0]
df['longitude']=df['point'].str[1]
df_backup=df.copy()
df.head(5)

,ein,name_org,tax_yr,phone,website,address,formation_yr,gross_receipts,ntee_code_nccs_x,ntee_description_nccs,...,volunteer_tot_ct,gross_receipts:1,rev_tot_curyr,rev_less_expense_curyr,local_chapt,address_code,location,point,latitude,longitude
11,226052626,STEVENS ALUMNI ASSOCIATION,2011,+1 (201) 216-5163,WWW.ALUMNI.STEVENS-TECH.EDU,Castle Point On The Hudson,1958,956073,B84,Alumni Associations,...,300,956073,705616,-22150,no,"Castle Point On The Hudson, Hoboken, Nj","(Castle Point on Hudson, Sinatra Drive, Hoboke...","(40.74435205, -74.02378415629497, 0.0)",40.744352,-74.023784
14,222891851,Columbian Arms Inc,2015,None,None,514 Madison Street,1987,896454,L21,Public Housing Facilities,...,None,896454,-40166,-40166,no,"514 Madison Street, Hoboken, Nj","(514, Madison Street, Hoboken, Hudson County, ...","(40.743918, -74.03775914285714, 0.0)",40.743918,-74.037759
64,223517040,HOBOKEN CHARTER SCHOOL INC,2016,+1 (201) 963-0222,WWW.HOBOKENCS.ORG,713 Washington Street,1997,5346175,None,None,...,None,5346175,5346175,276176,no,"713 Washington Street, Hoboken, Nj","(713, Washington Street, Hoboken, Hudson Count...","(40.74615489090909, -74.02808611818182, 0.0)",40.746155,-74.028086
96,221801849,HOPES COMMUNITY ACTION PARTNERSHIP,2016,+1 (855) 654-6737,WWW.HOPES.ORG,301 Garden Street,1964,16748872,S20,None,...,788,16748872,15513189,1916145,no,"301 Garden Street, Hoboken, Nj","(301, Garden Street, Hoboken, Hudson County, N...","(40.74038, -74.031745, 0.0)",40.740380,-74.031745
107,200596839,Party With Purpose Inc,2014,+1 (201) 988-7175,None,Po Box 1045,2002,183780,T31,Community Foundations,...,100,183780,183780,55830,no,"Party With Purpose Inc, Hoboken, Nj",None,None,NaN,NaN


Even though NTEE code is very detailed, shown by numbers after the alphabet, I am only interested the main divisions I coded the empty values as Z, which is consistent with the original NTEE code.

In [11]:
df['ntee_code_nccs_x']=df['ntee_code_nccs_x'].str[0]
df['ntee_code_nccs_x']=df['ntee_code_nccs_x'].apply(lambda code: 'Z' if pd.isna(code)==True else code)
uniq_code=list(set(df['ntee_code_nccs_x']))

# Plotting the Map
To plot, I used folium as this was fairly easy to use, and you do not need any additional permission. You can see the representation of the map can be improved, which will be discussed later, but I think this is a great way to explore geospatial data.

In [12]:
# Included colors so each marker can be color-coded
colors=['darkred', 'blue', 'green', 'purple', 'orange','lightgray', 'gray',
        'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue','black']

def codecolors(counter):
    for i in range(0,len(colors)):
        if counter == uniq_code[i]:
            return colors[i]
    
df['color'] = df['ntee_code_nccs_x'].apply(codecolors)
df.head(5)

,ein,name_org,tax_yr,phone,website,address,formation_yr,gross_receipts,ntee_code_nccs_x,ntee_description_nccs,...,gross_receipts:1,rev_tot_curyr,rev_less_expense_curyr,local_chapt,address_code,location,point,latitude,longitude,color
11,226052626,STEVENS ALUMNI ASSOCIATION,2011,+1 (201) 216-5163,WWW.ALUMNI.STEVENS-TECH.EDU,Castle Point On The Hudson,1958,956073,B,Alumni Associations,...,956073,705616,-22150,no,"Castle Point On The Hudson, Hoboken, Nj","(Castle Point on Hudson, Sinatra Drive, Hoboke...","(40.74435205, -74.02378415629497, 0.0)",40.744352,-74.023784,darkred
14,222891851,Columbian Arms Inc,2015,None,None,514 Madison Street,1987,896454,L,Public Housing Facilities,...,896454,-40166,-40166,no,"514 Madison Street, Hoboken, Nj","(514, Madison Street, Hoboken, Hudson County, ...","(40.743918, -74.03775914285714, 0.0)",40.743918,-74.037759,green
64,223517040,HOBOKEN CHARTER SCHOOL INC,2016,+1 (201) 963-0222,WWW.HOBOKENCS.ORG,713 Washington Street,1997,5346175,Z,None,...,5346175,5346175,276176,no,"713 Washington Street, Hoboken, Nj","(713, Washington Street, Hoboken, Hudson Count...","(40.74615489090909, -74.02808611818182, 0.0)",40.746155,-74.028086,lightgray
96,221801849,HOPES COMMUNITY ACTION PARTNERSHIP,2016,+1 (855) 654-6737,WWW.HOPES.ORG,301 Garden Street,1964,16748872,S,None,...,16748872,15513189,1916145,no,"301 Garden Street, Hoboken, Nj","(301, Garden Street, Hoboken, Hudson County, N...","(40.74038, -74.031745, 0.0)",40.740380,-74.031745,blue
107,200596839,Party With Purpose Inc,2014,+1 (201) 988-7175,None,Po Box 1045,2002,183780,T,Community Foundations,...,183780,183780,55830,no,"Party With Purpose Inc, Hoboken, Nj",None,None,NaN,NaN,orange


In [13]:
df

,ein,name_org,tax_yr,phone,website,address,formation_yr,gross_receipts,ntee_code_nccs_x,ntee_description_nccs,...,gross_receipts:1,rev_tot_curyr,rev_less_expense_curyr,local_chapt,address_code,location,point,latitude,longitude,color
11,226052626,STEVENS ALUMNI ASSOCIATION,2011,+1 (201) 216-5163,WWW.ALUMNI.STEVENS-TECH.EDU,Castle Point On The Hudson,1958,956073,B,Alumni Associations,...,956073,705616,-22150,no,"Castle Point On The Hudson, Hoboken, Nj","(Castle Point on Hudson, Sinatra Drive, Hoboke...","(40.74435205, -74.02378415629497, 0.0)",40.744352,-74.023784,darkred
14,222891851,Columbian Arms Inc,2015,None,None,514 Madison Street,1987,896454,L,Public Housing Facilities,...,896454,-40166,-40166,no,"514 Madison Street, Hoboken, Nj","(514, Madison Street, Hoboken, Hudson County, ...","(40.743918, -74.03775914285714, 0.0)",40.743918,-74.037759,green
64,223517040,HOBOKEN CHARTER SCHOOL INC,2016,+1 (201) 963-0222,WWW.HOBOKENCS.ORG,713 Washington Street,1997,5346175,Z,None,...,5346175,5346175,276176,no,"713 Washington Street, Hoboken, Nj","(713, Washington Street, Hoboken, Hudson Count...","(40.74615489090909, -74.02808611818182, 0.0)",40.746155,-74.028086,lightgray
96,221801849,HOPES COMMUNITY ACTION PARTNERSHIP,2016,+1 (855) 654-6737,WWW.HOPES.ORG,301 Garden Street,1964,16748872,S,None,...,16748872,15513189,1916145,no,"301 Garden Street, Hoboken, Nj","(301, Garden Street, Hoboken, Hudson County, N...","(40.74038, -74.031745, 0.0)",40.740380,-74.031745,blue
107,200596839,Party With Purpose Inc,2014,+1 (201) 988-7175,None,Po Box 1045,2002,183780,T,Community Foundations,...,183780,183780,55830,no,"Party With Purpose Inc, Hoboken, Nj",None,None,NaN,NaN,orange
110,10873064,HUDSON HEALTHCARE INC,2010,+1 (201) 418-2249,WWW.HOBOKENUMC.COM,308 Willow Ave,2007,130061664,E,Public Health Program (Includes General Health...,...,130061664,130025839,-23106291,no,"308 Willow Ave, Hoboken, Nj","(Hoboken University Medical Center, 308, Willo...","(40.741482500000004, -74.03397040924716, 0.0)",40.741483,-74.033970,darkgreen
111,300060431,LITTLE LEAGUE BASEBALL INC 2300706 HOBOKEN LIT...,2011,+1 (201) 697-1995,LEAGUELINEUP.COM,25 Fifth Street The Fieldhouse,1990,22030,N,"Baseball, Softball",...,22030,22030,-2184,no,"25 Fifth Street, Hoboken, Nj",None,None,NaN,NaN,pink
155,462360340,THE WATERFRONT PROJECT INC,2015,+1 (201) 308-3986,WWW.THEWATERFRONTPROJECT.ORG,404 Hudson Street,2013,90696,R,Minority Rights,...,90696,90696,10449,no,"404 Hudson Street, Hoboken, Nj","(Saints Peter and Paul RC Church, 404, Hudson ...","(40.74128815, -74.02891129487821, 0.0)",40.741288,-74.028911,cadetblue
177,912073437,ALL SAINTS COMMUNITY SERVICE AND DEVELOPMENT C...,2014,+1 (201) 792-0340,ASCSDC.ORG,601-603 Jackson Street,1995,1034016,S,"Community, Neighborhood Development, Improveme...",...,1034016,1013372,-260140,no,"601-603 Jackson Street, Hoboken, Nj","(Jubilee Center, 601-603, Jackson Street, Hobo...","(40.74537425, -74.03886447905842, 0.0)",40.745374,-74.038864,blue
213,223536228,FRIENDS OF THE ELYSIAN CHARTER SCHOOL OF HOBOKEN,2015,+1 (201) 876-0102,None,1460 Garden Street,1998,437918,B,Education,...,437918,347354,-154668,no,"1460 Garden Street, Hoboken, Nj","(1460, Garden Street, Hoboken, Hudson County, ...","(40.75398251482986, -74.02781848704882, 0.0)",40.753983,-74.027818,darkred


Some of the addresses were not coded because the original addresses are a little strange. For example, searching Party with Purpose Inc on the Internet shows that they throws events at different places, and they may not have a physical office. Thus, the longitude and latitude of these places will be filled with the position of the center of the city.

In [14]:
import folium

df['longitude']=df['longitude'].fillna(-74.032694)
df['latitude']=df['latitude'].fillna(40.744070)
m = folium.Map(location=[40.744070, -74.032694],tiles='CartoDB positron', zoom_start=15)

To create a legend, I have copied down what each alphabet represents and created a label in the data frame.

In [15]:
ntee_label=['Education', 'Philanthropy, Voluntarism & Grantmaking Foundations', 'Arts, Culture & Humanities',
            'Employment','Health Care','Civil Rights, Social Action & Advocacy',
            'Voluntary Health Associations & Medical Disciplines',
            'Unknown/Not Specified', 'Housing & Shelter',
            'Community Improvement & Capacity Building','Human Services',
            'Religion-Related','Mutual & Membership Benefit','Recreation & Sports'
            ]
def labelcode(counter):
    for i in range(0,len(ntee_label)):
        if counter == uniq_code[i]:
            return ntee_label[i]
df['ntee_label'] = df['ntee_code_nccs_x'].apply(labelcode)  
df['name_org']=df['name_org'].str.title() 

# Filling in empty values...
df['address']=df['address'].str[:-13]
df['phone']=df['phone'].apply(lambda x: 'N/A' if pd.isna(x)==True else x)

To label the markers, I have added the name of the organization, address, and phone number. The original folium does not have an easy way to implement legend on the map, so I used html to created a legend. Because of this, the colors are a little off. The general folium plotting was inspired by the sites below: 
- https://georgetsilva.github.io/posts/mapping-points-with-folium/
- https://geopy.readthedocs.io/en/stable/#module-geopy.geocoders
- https://python-visualization.github.io/folium/modules.html

In [16]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(m)
df.apply(lambda row:folium.Marker(
    location=[row['latitude'], row['longitude']], 
    popup=row['name_org']+'  '+row['address'] +'  '+row['phone'], max_width=500,min_width=400,
    icon=folium.Icon(color=row['color'], icon='hand-holding-heart')).add_to(marker_cluster),
         axis=1)

legend_html =   '''
                <div style="position: fixed; 
                            bottom: 40px; left: 40px; width: 410px; height:380px; 
                            border:2px solid grey; z-index:9999; font-size:12px;
                            ">&nbsp; Ntee code <br>
                              &nbsp; Education &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkred"></i><br>
                              &nbsp; Philan., Volunt. & Grantmaking Foundations &nbsp; <i class="fa fa-map-marker fa-2x" style="color:dodgerblue"></i><br>
                              &nbsp; Arts, Culture & Humanities &nbsp; <i class="fa fa-map-marker fa-2x" style="color:forestgreen"></i><br>
                              &nbsp; Employment &nbsp; <i class="fa fa-map-marker fa-2x" style="color:mediumorchid"></i><br>
                              &nbsp; Health Care &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
                              &nbsp; Civil Rights, Social Action & Adv. &nbsp; <i class="fa fa-map-marker fa-2x" style="color:gray"></i><br>
                              &nbsp; Volunt. Health Asso. & Medical Discipl. &nbsp; <i class="fa fa-map-marker fa-2x" style="color:dimgray"></i><br>
                              &nbsp; Unknown/Not Specified &nbsp; <i class="fa fa-map-marker fa-2x" style="color:steelblue"></i><br>
                              &nbsp; Housing & Shelter &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkolivegreen"></i><br>
                              &nbsp; Community Improv. & Capacity Building &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkcyan"></i><br>
                              &nbsp; Human Services &nbsp; <i class="fa fa-map-marker fa-2x" style="color:indigo"></i><br>
                              &nbsp; Religion-Related &nbsp; <i class="fa fa-map-marker fa-2x" style="color:magenta"></i><br>
                              &nbsp; Mutual & Member. Benefit &nbsp; <i class="fa fa-map-marker fa-2x" style="color:lightskyblue"></i><br>
                              &nbsp; Recreation & Sports &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i>
                </div>
                ''' 

m.get_root().html.add_child(folium.Element(legend_html))

m.save('hoboken_map.html')

We still see 8 addresses that are not properly coded. This is because some of the addresses that are listed in words (eg, Fifth street) are not properly coded. We will code them using a dictionary.

In [17]:
m

You can see that it has now been coded properly.

In [18]:
text_dict = {'First':'1st', 'Second':'2nd', 'Third':'3rd', 'Fourth':'4th', 'Fifth':'5th', 'Sixth': '6th', 'Seventh':'7th', 'Eighth':'8th', 'Ninth':'9th'}
for word, initial in text_dict.items():
    df['address_code'] = df['address_code'].apply(lambda x: x.replace(word, initial))
df['address_code']

11            Castle Point On The Hudson, Hoboken, Nj
14                    514 Madison Street, Hoboken, Nj
64                 713 Washington Street, Hoboken, Nj
96                     301 Garden Street, Hoboken, Nj
107               Party With Purpose Inc, Hoboken, Nj
110                       308 Willow Ave, Hoboken, Nj
111                        25 5th Street, Hoboken, Nj
155                    404 Hudson Street, Hoboken, Nj
177               601-603 Jackson Street, Hoboken, Nj
213                   1460 Garden Street, Hoboken, Nj
224                         601 Park Ave, Hoboken, Nj
232                        106 Hudson St, Hoboken, Nj
249                       308 Willow Ave, Hoboken, Nj
255           C/O P Capra 51 Harrison St, Hoboken, Nj
259        Hoboken Historical Museum Inc, Hoboken, Nj
265                    301 Garden Street, Hoboken, Nj
278                    301 Garden Street, Hoboken, Nj
293                          251 10Th St, Hoboken, Nj
294             926 Castle P

Now we are geocoding the address code again to see if we can plot most of the addresses appropriately. The legend is optimized for html, so we can ignore for now. 

In [19]:
df['location'] = df['address_code'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df['latitude']=df['point'].str[0]
df['longitude']=df['point'].str[1]
df['longitude']=df['longitude'].fillna(-74.032694)
df['latitude']=df['latitude'].fillna(40.744070)

marker_cluster = MarkerCluster().add_to(m)
df.apply(lambda row:folium.Marker(
    location=[row['latitude'], row['longitude']], 
    popup=row['name_org']+'  '+row['address'] +'  '+row['phone'], max_width=500,min_width=400,
    icon=folium.Icon(color=row['color'], icon='human')).add_to(marker_cluster),
         axis=1)
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 40px; left: 40px; width: 410px; height:380px; 
                            border:2px solid grey; z-index:9999; font-size:12px;
                            ">&nbsp; Ntee code <br>
                              &nbsp; Education &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkred"></i><br>
                              &nbsp; Philan., Volunt. & Grantmaking Foundations &nbsp; <i class="fa fa-map-marker fa-2x" style="color:dodgerblue"></i><br>
                              &nbsp; Arts, Culture & Humanities &nbsp; <i class="fa fa-map-marker fa-2x" style="color:forestgreen"></i><br>
                              &nbsp; Employment &nbsp; <i class="fa fa-map-marker fa-2x" style="color:mediumorchid"></i><br>
                              &nbsp; Health Care &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
                              &nbsp; Civil Rights, Social Action & Adv. &nbsp; <i class="fa fa-map-marker fa-2x" style="color:gray"></i><br>
                              &nbsp; Volunt. Health Asso. & Medical Discipl. &nbsp; <i class="fa fa-map-marker fa-2x" style="color:dimgray"></i><br>
                              &nbsp; Unknown/Not Specified &nbsp; <i class="fa fa-map-marker fa-2x" style="color:steelblue"></i><br>
                              &nbsp; Housing & Shelter &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkolivegreen"></i><br>
                              &nbsp; Community Improv. & Capacity Building &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkcyan"></i><br>
                              &nbsp; Human Services &nbsp; <i class="fa fa-map-marker fa-2x" style="color:indigo"></i><br>
                              &nbsp; Religion-Related &nbsp; <i class="fa fa-map-marker fa-2x" style="color:magenta"></i><br>
                              &nbsp; Mutual & Member. Benefit &nbsp; <i class="fa fa-map-marker fa-2x" style="color:lightskyblue"></i><br>
                              &nbsp; Recreation & Sports &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i>
                </div>
                ''' 

m.get_root().html.add_child(folium.Element(legend_html))

m.save('hoboken_map.html')
m

# Future improvements
1. As we can see, some of the NTEE codes are not relevant for our uses. Thus, we can bin them or delete some categories, which will help with the colors and legend size.
2. Address and phone number boxes are too small; based on the forum, I should be able to fix the width. However, it doesn't seem to be the case. This can be revisited next time.

In [20]:
pip install git+https://github.com/python-visualization/folium@master

  Cloning https://github.com/python-visualization/folium (to revision master) to c:\users\andrea\appdata\local\temp\pip-req-build-gbxd017v
  Created wheel for folium: filename=folium-0.10.1+8.g4affacb-py2.py3-none-any.whl size=94503 sha256=27ab6731e7d5c486d38e788f1fdd433ab8fa367c73541405f85de771d4b5402d
  Stored in directory: C:\Users\Andrea\AppData\Local\Temp\pip-ephem-wheel-cache-86vb6m86\wheels\68\e4\23\2a744cccbfacdb83fbbe1025baa03503bba83167a7c4d12993
Successfully built folium
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/python-visualization/folium 'C:\Users\Andrea\AppData\Local\Temp\pip-req-build-gbxd017v'


In [26]:
df['address_code']=df['address_code'].str[:-13]
m = folium.Map(location=[40.744070, -74.032694],tiles='CartoDB positron', zoom_start=15)
marker_cluster = MarkerCluster().add_to(m)
df.apply(lambda row:folium.Marker(
    location=[row['latitude'], row['longitude']], 
    popup=row['name_org']+'  '+row['address_code'] +'  '+row['phone'], max_width=1000,min_width=900,
    icon=folium.Icon(color=row['color'], icon='hand-holding-heart')).add_to(marker_cluster),
         axis=1)
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 40px; left: 40px; width: 370px; height:380px; 
                            border:2px solid grey; z-index:9999; font-size:12px;
                            ">&nbsp; Ntee code <br>
                              &nbsp; Education &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkred"></i><br>
                              &nbsp; Philan., Volunt. & Grantmaking Foundations &nbsp; <i class="fa fa-map-marker fa-2x" style="color:dodgerblue"></i><br>
                              &nbsp; Arts, Culture & Humanities &nbsp; <i class="fa fa-map-marker fa-2x" style="color:forestgreen"></i><br>
                              &nbsp; Employment &nbsp; <i class="fa fa-map-marker fa-2x" style="color:mediumorchid"></i><br>
                              &nbsp; Health Care &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
                              &nbsp; Civil Rights, Social Action & Adv. &nbsp; <i class="fa fa-map-marker fa-2x" style="color:gray"></i><br>
                              &nbsp; Volunt. Health Asso. & Medical Discipl. &nbsp; <i class="fa fa-map-marker fa-2x" style="color:dimgray"></i><br>
                              &nbsp; Unknown/Not Specified &nbsp; <i class="fa fa-map-marker fa-2x" style="color:steelblue"></i><br>
                              &nbsp; Housing & Shelter &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkolivegreen"></i><br>
                              &nbsp; Community Improv. & Capacity Building &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkcyan"></i><br>
                              &nbsp; Human Services &nbsp; <i class="fa fa-map-marker fa-2x" style="color:indigo"></i><br>
                              &nbsp; Religion-Related &nbsp; <i class="fa fa-map-marker fa-2x" style="color:magenta"></i><br>
                              &nbsp; Mutual & Member. Benefit &nbsp; <i class="fa fa-map-marker fa-2x" style="color:lightskyblue"></i><br>
                              &nbsp; Recreation & Sports &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i>
                </div>
                ''' 

m.get_root().html.add_child(folium.Element(legend_html))

m.save('hoboken_map.html')
w = sa.create_engine(r'sqlite:///D:\Program_practices\Charity\geotag.sqlite')
df=df.drop(columns=['location', 'point'])
df.to_sql('hoboken_map', w, if_exists='replace', index=False)
m